This notebook look at all the balis found in document and extract statisics about them.

In [13]:
%matplotlib inline
from xml.dom import minidom
from tika import parser
import glob
import os
import tika
from collections import Counter
from xml.dom import Node
from tqdm import tqdm_notebook as tqdm
from utils import *

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
files = glob.glob('../docs/**/*', recursive=True)
files = filter(lambda file: not os.path.isdir(file), files)
file_counter = Counter(map(lambda x: x.split('.')[-1], files))
file_counter.most_common(5)

[('gml', 431), ('pdf', 249), ('pptx', 82), ('doc', 46), ('ppt', 34)]

In [4]:
tika.initVM()

In [5]:
c = Counter()
def flat(node):
    for x in node.childNodes:
        if x.nodeType == Node.ELEMENT_NODE:
            #print(x.tagName)
            #print(x.attributes.items())
            c.update([x.tagName])
            for g in flat(x):
                yield g
            yield x.tagName
for file in glob.glob('../docs/**/*.doc', recursive=True):
    try:
        parsed = parser.from_file(file, xmlContent=True)
        # print(parsed['content'])
        xmldoc = minidom.parseString(parsed['content'])
        c.update(flat(xmldoc))
    except Exception as e:
        pass
        #print(f'Exception {e} with {file}')

2018-11-28 09:10:06,582 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.19/tika-server-1.19.jar.md5 to /tmp/tika-server.jar.md5.
2018-11-28 09:10:07,078 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.19/tika-server-1.19.jar to /tmp/tika-server.jar.
2018-11-28 09:10:21,201 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...


In [7]:
c#['table']

Counter({'a': 206,
         'b': 14782,
         'body': 48,
         'div': 70,
         'h1': 12,
         'h2': 2,
         'h3': 22,
         'head': 48,
         'html': 48,
         'i': 548,
         'meta': 2348,
         'p': 11646,
         'table': 28,
         'tbody': 28,
         'td': 1320,
         'title': 48,
         'tr': 350,
         'u': 740})

# Doc2Graph
Construct graph where each node is connected to next node and previous node, his direct parent, and his direct children. In order not to lose the sense of the graph each ending node of a block are conected to a special kind of node beginingNode and endingNode 

In [8]:
import networkx as nx

In [9]:
node_id = 1
parent = 0
def rec_graph(xml_node):
    global node_id
    global parent
    root = node_id
    G = nx.Graph()
    G.add_node(node_id, tag='Begining')
    G.add_edge(parent, node_id, relation='parent')
    prev_node_id = node_id
    node_id+=1
    for x in xml_node.childNodes:
        if x.nodeType != Node.TEXT_NODE:
            #print(type(x.tagName))
            G.add_node(node_id, tag=x.tagName)
            G.add_edge(prev_node_id, node_id, relation='siblings')
            G.add_edge(node_id, parent, relation='parent')
            prev_node_id = node_id
            node_id+=1
            if x.nodeType == Node.ELEMENT_NODE:
                prev_parent = parent
                parent = node_id - 1
                g_child, root = rec_graph(x)
                G = nx.compose(G, g_child)
                parent = prev_parent
        else:
            G.add_node(node_id, tag='text', value=x.data)
            G.add_edge(prev_node_id, node_id, relation='siblings')
            G.add_edge(node_id, parent, relation='parent')
            prev_node_id = node_id
            node_id+=1
    G.add_node(node_id, tag='Ending')
    G.add_edge(node_id, parent, relation='parent')
    G.add_edge(node_id, prev_node_id, relation='siblings')
    node_id+=1
    return G, root
G, root = rec_graph(xmldoc)

In [ ]:
# This cell takes time to be executed
for file in glob.glob('../docs/**/*.pptx', recursive=True):
    #print(file)
    try:
        parsed = parser.from_file(file, xmlContent=True)
        # print(parsed['content'])
        # print("============================================")
        xmls = split_str2xml(parsed['content'])
        for i, xml in enumerate(xmls):
            xmldoc = minidom.parseString(xml)
            G, root = rec_graph(xmldoc)
            #print(xml)
            #print("==================")
            nx.write_gml(G, f'{file}-part{i}.gml')
    except Exception as e:
        raise e
        print(f"error with: {file}")
        #pass
        #print(f'Exception {e} with {file}')

In [ ]:
nx.draw(G)